<h1> Assignment 2: deep learning on images </h1>
<p> In this assignment, more than 11 000 pictures will be used to create a model able to recognize the food content on a picture. The images used are scrapped from instagram having the hashtag #recipe. The codes for scrapping the instagram pictures are <a href="http://seppe.net/aa/assignment2/instagram-scraper.py">given here</a>.
The actual pictures can be downloaded from <a href="https://drive.google.com/file/d/1RnQa3nrjHGqj6fM_St9GU_4CrFV0EaZB/view">this google drive</a>. Next to the raw images we have meta-data for each image in the google drive. This is in a csv file available in the repository "recipes.csv". </p>
<p> We will try to stick to a typical Data Science workflow for this assignment. Let's get started !  <p>

<h2> 1. Loading images </h2>
We start by loading the data described above. The challenge is to load more than 11 000 raw images (6GB) in our Python world into a format that we can work with (pixels).

In [1]:
from tensorflow.keras.preprocessing import image
import numpy as np
import pandas as pd
import os
os.chdir('C:/Users/david/Documents/0_kul/1_ma_stds_1/1_advanced_analytics/assignment_2/data/images')

In [22]:
# creating function to extract pixel from png
def get_images(path):
    X = []
    for img_path in path:
        img = image.load_img(img_path,target_size=(100,100)) # it is easier to work with one size. we set it to 200x200
        x = image.img_to_array(img)
        X.append(x)
    return np.array(X)

Unfortunaly calling this new defined function gives us an error. We quickly find out that some images in the dataset are empty, and thus need to be removed. It is easily done manually. On top of that, we add the undesired id of the images in a list to remove them  from the meta-data.

In [23]:
# Make a dataframe with unwanted rows/data.
empty = ['CMbgB5tjSJa', 'CMbJFC-F3Tz',' CMmV2A2sBvY', 'CMMyIsZgX31', 'CMO5_RRDV5z',
'CMO98R3n1M3', 'CMO967MAl3V','CMOOv-qh1fN', 'CMP4fSDrLhn', 'CMT_uYYFJIp', 'CMTz1OZqDAG','CMmV2A2sBvY']

In [24]:
# loading the label dataset
df = pd.read_csv('C:/Users/david/Documents/0_kul/1_ma_stds_1/1_advanced_analytics/assignment_2/data/recipes.csv',delimiter=';')
# Use the index of this unwanted dataframe to drop the rows from the original dataframe.
df = df.drop(df_undesired.index,axis=0)
df.shape

(11685, 104)

In [25]:
# creating the path
my_path = df.photo_id.values+'.png'
my_path

array(['CMPELQqh_nh.png', 'CMPEHqppR-t.png', 'CMPEBY9Buko.png', ...,
       'CMsOrPOrSkl.png', 'CMsOnHuFxoF.png', 'CMsOZ0MpBbu.png'],
      dtype=object)

In [26]:
# Extracting the pixels from the images
pixel = get_images(my_path)

In [27]:
print(type(pixel))
print(pixel.shape)

<class 'numpy.ndarray'>
(11685, 100, 100, 3)


<h2> 2. Saving the files </h2>
In order to ease the process of working with the data we save the pixel format.

In [35]:
np.save('image_pixel', pixel) # saving it
loaded_array = np.load('image_pixel.npy') # loading it

<h2> 3. Preprocessing the files </h2>
There a few things we need to take care of before we fit a deep learning model. We have a heavy dataset.
Deciding which tags we want to use. perhaps merge some.
downscale the images. (which we kind of did already) or image augmentation.

In [42]:
# loading the pixel format dataset
x = np.load('C:/Users/david/Documents/0_kul/1_ma_stds_1/1_advanced_analytics/assignment_2/data/processed/image_pixel.npy') # loading it

MemoryError: Unable to allocate 1.31 GiB for an array with shape (350550000,) and data type float32

<h3> 3.1 New Tags </h3>

In [28]:
df['tag_keto_concept'] = df['tag_keto'] + df['tag_ketodiet'] + df['tag_lowcarb']
df['tag_italy_concept'] = df['tag_italy'] + df['tag_italia'] + df['tag_italianfood']
df['tag_oat_concept'] = df['tag_oats'] + df['tag_oatmeal']
df['tag_vegan_concept'] = df['tag_vegan'] + df['tag_veganfood'] + df['tag_plantbased'] + df['tag_veganrecipe']
df['tag_pasta_concept'] = df['tag_pasta'] + df['tag_pastalover']
df['tag_sweet_concept'] = df['tag_desserts'] + df['tag_sweets'] + df['tag_dessert'] + df['tag_sweettooth']
df['tag_vegetarian_concept'] = df['tag_vegetarian'] + df['tag_vegetarianrecipes'] + df['tag_veggie']
df['tag_egg_concept'] = df['tag_egg'] + df['tag_eggs']
df['tag_meat_concept'] = df['tag_meat'] + df['tag_beef'] + df['tag_bbq']
df['tag_diet_concept'] = df['tag_diet'] + df['tag_weightloss'] + df['tag_fitness'] + df['tag_healthy']
df['tag_chocolate_concept'] = df['tag_chocolat'] + df['tag_chocolate']
df['tag_bake_concept'] = df['tag_baking'] + df['tag_cake'] + df['tag_bake'] + df['tag_bakery'] + df['tag_cakes'] + df['tag_pastry'] + df['tag_baker']
df['tag_breakfast_concept'] = df['tag_breakfast'] + df['tag_breakfastideas']
df['tag_fish_concept'] = df['tag_fish'] + df['tag_salmon']

In [29]:
remove = ['tag_keto','tag_ketodiet','tag_lowcarb','tag_italy','tag_italia',
'tag_italianfood','tag_oats','tag_oatmeal','tag_vegan','tag_veganfood',
'tag_plantbased','tag_veganrecipe','tag_pasta','tag_pastalover','tag_desserts',
'tag_sweets','tag_dessert','tag_sweettooth','tag_vegetarian','tag_vegetarianrecipes',
'tag_veggie','tag_egg','tag_eggs','tag_meat','tag_beef',
'tag_bbq','tag_diet','tag_weightloss','tag_fitness','tag_healthy',
'tag_chocolat','tag_chocolate','tag_baking','tag_cake','tag_bake',
'tag_bakery','tag_cakes','tag_pastry','tag_baker','tag_breakfast',
'tag_breakfastideas','tag_fish','tag_salmon']
df.drop(remove,axis=1)

photo_id                                       url  \
0      CMPELQqh_nh  https://www.instagram.com/p/CMPELQqh_nh/   
1      CMPEHqppR-t  https://www.instagram.com/p/CMPEHqppR-t/   
2      CMPEBY9Buko  https://www.instagram.com/p/CMPEBY9Buko/   
3      CMPEAvhF9i1  https://www.instagram.com/p/CMPEAvhF9i1/   
4      CMPD73jJssJ  https://www.instagram.com/p/CMPD73jJssJ/   
...            ...                                       ...   
11691  CMsOtSZhtSx  https://www.instagram.com/p/CMsOtSZhtSx/   
11692  CMsOstJAXEJ  https://www.instagram.com/p/CMsOstJAXEJ/   
11693  CMsOrPOrSkl  https://www.instagram.com/p/CMsOrPOrSkl/   
11694  CMsOnHuFxoF  https://www.instagram.com/p/CMsOnHuFxoF/   
11695  CMsOZ0MpBbu  https://www.instagram.com/p/CMsOZ0MpBbu/   

                   may_be  likes            timestamp           scraped_at  \
0      dessert and indoor      3  2021-03-10T10:47:18  2021-03-10T11:48:52   
1        pasta and indoor      3  2021-03-10T10:46:48  2021-03-10T11:48:59   
2         cake and indoor      2  2021-03-10T10:45:57  2021-03-10T11:49:08   
3                    food      1  2021-03-10T10:45:52  2021-03-10T11:49:12   
4                 dessert      2  2021-03-10T10:45:12  2021-03-10T11:49:16   
...                   ...    ...                  ...                  ...   
11691  dessert and indoor     10  2021-03-21T18:59:16  2021-03-22T10:08:38   
11692          strawberry     16  2021-03-21T18:37:13  2021-03-22T10:08:42   
11693                food     15  2021-03-21T18:56:11  2021-03-22T10:08:50   
11694                food     23  2021-03-21T21:31:55  2021-03-22T10:09:03   
11695     food and indoor     21  2021-03-21T18:43:47  2021-03-22T10:09:09   

        diff  tag_tasty  tag_dinner  tag_lunch  ...  tag_pasta_concept  \
0       1.03          0           1          0  ...                  0   
1       1.04          0           0          0  ...                  0   
2       1.05          0           0          0  ...                  0   
3       1.06          1           0          0  ...                  0   
4       1.07          0           0          0  ...                  0   
...      ...        ...         ...        ...  ...                ...   
11691  15.16          0           0          0  ...                  0   
11692  15.52          0           0          0  ...                  0   
11693  15.21          0           0          0  ...                  0   
11694  12.62          0           0          0  ...                  0   
11695  15.42          0           0          0  ...                  0   

       tag_sweet_concept  tag_vegetarian_concept  tag_egg_concept  \
0                      0                       0                0   
1                      0                       2                0   
2                      0                       0                0   
3                      0                       0                0   
4                      2                       0                0   
...                  ...                     ...              ...   
11691                  0                       0                0   
11692                  1                       0                0   
11693                  0                       0                0   
11694                  0                       0                0   
11695                  0                       0                1   

       tag_meat_concept  tag_diet_concept  tag_chocolate_concept  \
0                     0                 0                      0   
1                     0                 0                      0   
2                     0                 1                      0   
3                     0                 0                      0   
4                     0                 0                      1   
...                 ...               ...                    ...   
11691                 0                 0                      0   
11692                 0                 0 

In [30]:
df['tag_keto_concept']= np.where(df['tag_keto_concept']<=1, 0, 1)
df['tag_italy_concept']= np.where(df['tag_italy_concept']<=1, 0, 1)
df['tag_oat_concept']= np.where(df['tag_oat_concept']<=1, 0, 1)
df['tag_vegan_concept']= np.where(df['tag_vegan_concept']<=1, 0, 1)
df['tag_pasta_concept']= np.where(df['tag_pasta_concept']<=1, 0, 1)
df['tag_sweet_concept']= np.where(df['tag_sweet_concept']<=1, 0, 1)
df['tag_vegetarian_concept']= np.where(df['tag_vegetarian_concept']<=1, 0, 1)
df['tag_egg_concept']= np.where(df['tag_egg_concept']<=1, 0, 1)
df['tag_meat_concept']= np.where(df['tag_meat_concept']<=1, 0, 1)
df['tag_diet_concept']= np.where(df['tag_diet_concept']<=1, 0, 1)
df['tag_chocolate_concept']= np.where(df['tag_chocolate_concept']<=1, 0, 1)
df['tag_baking']= np.where(df['tag_baking']<=1, 0, 1)
df['tag_breakfast_concept']= np.where(df['tag_breakfast_concept']<=1, 0, 1)
df['tag_fish_concept']= np.where(df['tag_fish_concept']<=1, 0, 1)

<h3> 3.2 Downscale images </h3>

<h2> 4. Splitting into train, test and evaluation </h2>

In [31]:
necessary_columns = df.columns[7:]

In [37]:
# prepping y variable. necessary columns should be all the tags we want
y = df[necessary_columns]
print(y.shape)

(11685, 111)


In [38]:
from sklearn.model_selection import train_test_split

In [40]:
x_interim, x_eval, y_interim, y_eval = train_test_split(x,y,test_size=0.2,random_state=42)
x_train,x_test,y_train,y_test = train_test_split(x_interim,y_interim,test_size=0.4,random_state=42)

MemoryError: Unable to allocate 2.51 GiB for an array with shape (5608, 200, 200, 3) and data type float32

In [ ]:
# examine number of samples in train, test, and validation sets
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
print(x_eval.shape[0], 'eval samples')

<h2> 5. Model Building </h2>
We choose for a Sequential model, and add two layers of 15 and 5 respectively. Both layers have a kernel of size 3.

In [ ]:
model = Sequential()

In [ ]:
# Add a convolutional layer (15 units)
model.add(Conv2D(15,activation='relu',kernel_size=3,input_shape=(200,200,3)))
# Add another convolutional layer (5 units)
model.add(Conv2D(5,activation='relu',kernel_size=3))
# Flatten and feed to output layer
model.add(Flatten())
model.add(Dense(97, activation='softmax'))

In [ ]:
# Compile model
model.compile(optimizer='adam', 
              loss='binary_crossentropy', 
              metrics=['accuracy'])

<h2> 6. Fit the model to train data </h2>

In [ ]:
# Fit the model to training data 
model.fit(x_train,y_train, 
          validation_split=0.2, 
          epochs=3, batch_size=10)

<h2> 7. Evaluate the model with test data </h2>

In [ ]:
# Evaluate the model on test data
model.evaluate(x_test,y_test, batch_size=10)

<h2> 8. Interpreting our model </h2>

<h2> 9. Testing the model on some of our dishes </h2>

<h2> 10. Conclusion </h2>